In [1]:
#Kevin Madden 
#Due Date: 2/28/2022
#Professor: Dr. Hamed Tabkhi, PH.D.

import torch
from PIL import Image
from torchvision import transforms
import requests
from io import BytesIO
import numpy as np
import torch.optim as optim

#checking if I have GPU connected
torch.cuda.is_available()

True

In [2]:
#3.A
import csv
housing_path = "https://raw.githubusercontent.com/kmadden9/realTimeML/main/Housing_fixed.csv"
house_numpy = np.loadtxt(housing_path, dtype=np.float32, delimiter=",", skiprows=1)
house_numpy

array([[1.330e+07, 7.420e+03, 4.000e+00, 2.000e+00, 3.000e+00, 2.000e+00],
       [1.225e+07, 8.960e+03, 4.000e+00, 4.000e+00, 4.000e+00, 3.000e+00],
       [1.225e+07, 9.960e+03, 3.000e+00, 2.000e+00, 2.000e+00, 2.000e+00],
       ...,
       [1.750e+06, 3.620e+03, 2.000e+00, 1.000e+00, 1.000e+00, 0.000e+00],
       [1.750e+06, 2.910e+03, 3.000e+00, 1.000e+00, 1.000e+00, 0.000e+00],
       [1.750e+06, 3.850e+03, 3.000e+00, 1.000e+00, 2.000e+00, 0.000e+00]],
      dtype=float32)

In [3]:
col_list = next(csv.reader(open('/content/raw.githubusercontent.com/kmadden9/realTimeML/main/Housing_fixed.csv'), delimiter=','))
house_numpy.shape, col_list

((545, 6), ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking'])

In [4]:
house = torch.from_numpy(house_numpy)
house.shape, house.dtype

(torch.Size([545, 6]), torch.float32)

In [5]:
data = house[:, 1:]
target = house[:, 0]
data_mean = torch.mean(data, dim=0)
data_var = torch.var(data, dim=0)
target_mean = torch.mean(target, dim=0)
target_var = torch.var(target, dim=0)
data_normalized = (data - data_mean) / torch.sqrt(data_var)
target_normalized = (target - target_mean) / torch.sqrt(target_var)

area = torch.stack(tuple(data_normalized[:,0]))
bedrooms = torch.stack(tuple(data_normalized[:,1]))
bathrooms = torch.stack(tuple(data_normalized[:,2]))
stories = torch.stack(tuple(data_normalized[:,3]))
parking = torch.stack(tuple(data_normalized[:,4]))

In [6]:
target

tensor([13300000., 12250000., 12250000., 12215000., 11410000., 10850000.,
        10150000., 10150000.,  9870000.,  9800000.,  9800000.,  9681000.,
         9310000.,  9240000.,  9240000.,  9100000.,  9100000.,  8960000.,
         8890000.,  8855000.,  8750000.,  8680000.,  8645000.,  8645000.,
         8575000.,  8540000.,  8463000.,  8400000.,  8400000.,  8400000.,
         8400000.,  8400000.,  8295000.,  8190000.,  8120000.,  8080940.,
         8043000.,  7980000.,  7962500.,  7910000.,  7875000.,  7840000.,
         7700000.,  7700000.,  7560000.,  7560000.,  7525000.,  7490000.,
         7455000.,  7420000.,  7420000.,  7420000.,  7350000.,  7350000.,
         7350000.,  7350000.,  7343000.,  7245000.,  7210000.,  7210000.,
         7140000.,  7070000.,  7070000.,  7035000.,  7000000.,  6930000.,
         6930000.,  6895000.,  6860000.,  6790000.,  6790000.,  6755000.,
         6720000.,  6685000.,  6650000.,  6650000.,  6650000.,  6650000.,
         6650000.,  6650000.,  6629000

In [7]:
n_samples = target.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
train_indices, val_indices

(tensor([111, 504, 468, 240, 146,  31, 186, 364, 475, 250,  15, 264, 120, 442,
          89, 387, 180, 484, 375,  48,   4, 355, 301,  42, 306, 535, 476, 185,
         427, 234, 181,   6, 156,  57, 533, 405, 456, 270, 448,  50, 434, 487,
         490, 374, 299, 518, 276, 150, 478, 473, 522, 105, 324, 457,  56, 187,
         510, 444, 261, 542,  12,   9, 340, 233, 412, 483,  87, 297, 328, 397,
         227, 226, 417, 349, 486, 117, 342,  55,  41, 279, 193, 443, 127, 426,
         508, 329, 275, 410, 148, 123,  77, 303,  66, 513,  60, 500, 164, 360,
         289, 235, 296, 361, 350, 175, 119, 334, 317, 286, 525, 160,  28, 222,
         493, 454, 113, 316, 403, 523, 404, 116, 356, 414, 159,   5,  46,  97,
         248, 371, 437, 332, 241, 314, 194,  64, 394, 528,  69, 197, 424, 529,
         330, 256,  30,  62, 101,  45, 411, 441, 224, 173, 331, 368, 196, 362,
         265, 345, 452,   1, 280, 214, 492, 198, 106, 358, 450, 191, 536,  16,
         449, 353, 165, 451, 540,  73, 122, 271,  54

In [8]:
train_area = area[train_indices]
train_bedrooms = bedrooms[train_indices]
train_bathrooms = bathrooms[train_indices]
train_stories = stories[train_indices]
train_parking = parking[train_indices]

train_data = data_normalized[train_indices]
train_price = target_normalized[train_indices]

eval_area = area[val_indices]
eval_bedrooms = bedrooms[val_indices]
eval_bathrooms = bathrooms[val_indices]
eval_stories = stories[val_indices]
eval_parking = parking[val_indices]

eval_data = data_normalized[val_indices]
eval_price = target_normalized[val_indices]




In [9]:
def model(area, bedrooms, bathrooms, stories, parking, w1, w2, w3, w4, w5, b):
  return w5 * area + w4 * bedrooms + w3 * bathrooms + w2 * stories + w1 * parking + b

In [10]:
def loss_fn(price_pred, target_normalized):
  squared_diffs = (price_pred - target_normalized)**2
  return squared_diffs.mean()

In [11]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-5
optimizer = optim.SGD([params], lr=learning_rate)

In [12]:
price_pred = model(area, bedrooms, bathrooms, stories, parking, *params)
loss = loss_fn(price_pred, target_normalized)

loss.backward()
optimizer.step()

params

tensor([9.9997e-01, 9.9997e-01, 9.9997e-01, 9.9997e-01, 9.9998e-01, 3.0920e-12],
       requires_grad=True)

In [13]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

price_train = model(train_area, train_bedrooms, train_bathrooms, train_stories, train_parking, *params)
train_loss = loss_fn(price_train, train_price)

price_eval = model(eval_area, eval_bedrooms, eval_bathrooms, eval_stories, eval_parking, *params)
eval_loss = loss_fn(price_eval, eval_price)

optimizer.zero_grad()
train_loss.backward()
optimizer.step()

params

tensor([9.7378e-01, 9.6874e-01, 9.6824e-01, 9.6480e-01, 9.7262e-01, 4.9346e-04],
       requires_grad=True)

In [14]:
def training_loop(n_epochs, optimizer, params, train_area, train_bedrooms, train_bathrooms, train_stories, train_parking, train_price,
                  eval_area, eval_bedrooms, eval_bathrooms, eval_stories, eval_parking, eval_price):
  for epoch in range(1, n_epochs + 1):
    price_train = model(train_area, train_bedrooms, train_bathrooms, train_stories, train_parking, *params)
    train_loss = loss_fn(price_train, train_price)

    price_eval = model(eval_area, eval_bedrooms, eval_bathrooms, eval_stories, eval_parking, *params)
    eval_loss = loss_fn(price_eval, eval_price)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
      print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
        f" Validation loss {eval_loss.item():.4f}")
  
  return params

In [15]:
#2.b
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = .001
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
  n_epochs = 5000,
  optimizer = optimizer,
  params = params,
  train_area = train_area,
  train_bedrooms = train_bedrooms,
  train_bathrooms = train_bathrooms,
  train_stories = train_stories,
  train_parking = train_parking,
  train_price = train_price,
  eval_area = eval_area,
  eval_bedrooms = eval_bedrooms,
  eval_bathrooms = eval_bathrooms,
  eval_stories = eval_stories,
  eval_parking = eval_parking,
  eval_price = eval_price)

Epoch 1, Training loss 6.2892, Validation loss 5.0726
Epoch 2, Training loss 6.2426, Validation loss 5.0360
Epoch 3, Training loss 6.1965, Validation loss 4.9996
Epoch 500, Training loss 0.5568, Validation loss 0.5584
Epoch 1000, Training loss 0.4426, Validation loss 0.4609
Epoch 1500, Training loss 0.4377, Validation loss 0.4516
Epoch 2000, Training loss 0.4368, Validation loss 0.4480
Epoch 2500, Training loss 0.4365, Validation loss 0.4462
Epoch 3000, Training loss 0.4364, Validation loss 0.4454
Epoch 3500, Training loss 0.4364, Validation loss 0.4449
Epoch 4000, Training loss 0.4364, Validation loss 0.4447
Epoch 4500, Training loss 0.4364, Validation loss 0.4446
Epoch 5000, Training loss 0.4364, Validation loss 0.4445


tensor([ 0.1907,  0.2514,  0.2958,  0.0650,  0.3734, -0.0071],
       requires_grad=True)

In [16]:
import torch.nn as nn

linear_model = nn.Linear(5, 1)
linear_model(data_normalized)

tensor([[-7.5114e-01],
        [-2.2938e+00],
        [-5.6899e-02],
        [-5.5617e-01],
        [ 2.1855e-01],
        [-1.2069e+00],
        [-1.5605e+00],
        [-8.1923e-01],
        [ 3.1253e-01],
        [-9.0250e-01],
        [ 1.2808e+00],
        [-1.7574e+00],
        [-8.9563e-01],
        [-1.2131e+00],
        [-5.6359e-01],
        [ 2.2292e-02],
        [-8.8872e-01],
        [-4.1834e-01],
        [-7.9769e-01],
        [-6.5024e-01],
        [ 5.3464e-02],
        [-3.6474e-01],
        [ 5.4473e-01],
        [-9.0731e-01],
        [-2.1722e-01],
        [-7.9293e-01],
        [-9.7203e-01],
        [ 6.5875e-01],
        [-8.6142e-01],
        [-1.0408e+00],
        [-5.6001e-01],
        [ 2.6420e-01],
        [-1.1264e+00],
        [-1.6037e+00],
        [-2.2906e-01],
        [-6.2566e-01],
        [-5.8329e-01],
        [-6.1260e-01],
        [ 1.2599e-01],
        [-1.1313e+00],
        [ 1.5350e-01],
        [-9.2227e-01],
        [-6.9752e-01],
        [-1

In [17]:
linear_model.weight

Parameter containing:
tensor([[ 0.2999, -0.1944, -0.4471, -0.0693,  0.0897]], requires_grad=True)

In [18]:
linear_model.bias

Parameter containing:
tensor([-0.1977], requires_grad=True)

In [19]:
x = torch.ones(5)
linear_model(x)

tensor([-0.5189], grad_fn=<AddBackward0>)

In [20]:
x = torch.ones(10, 5)
linear_model(x)

tensor([[-0.5189],
        [-0.5189],
        [-0.5189],
        [-0.5189],
        [-0.5189],
        [-0.5189],
        [-0.5189],
        [-0.5189],
        [-0.5189],
        [-0.5189]], grad_fn=<AddmmBackward0>)

In [21]:
data_normalized = torch.tensor(data_normalized).unsqueeze(1)
target_normalized = torch.tensor(target_normalized).unsqueeze(1)


data_normalized.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


torch.Size([545, 1, 5])

In [42]:
target_normalized.shape

train_price = torch.tensor(train_price).unsqueeze(1)

eval_price = torch.tensor(eval_price).unsqueeze(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


torch.Size([109, 5])

In [43]:
linear_model = nn.Linear(5, 1)
optimizer = optim.SGD(
  linear_model.parameters(),
  lr=1e-2)

In [76]:
linear_model.parameters()

<generator object Module.parameters at 0x7f7380782250>

In [77]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[0.3737, 0.0647, 0.2960, 0.2515, 0.1902]], requires_grad=True),
 Parameter containing:
 tensor([-0.0072], requires_grad=True)]

In [202]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_data, train_price,
                  eval_data, eval_price):
  for epoch in range(1, n_epochs + 1):
    price_train = model(train_data)
    train_loss = loss_fn(price_train, train_price)

    price_eval = model(eval_data)
    eval_loss = loss_fn(price_eval, eval_price)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch == 1 or epoch % 50 == 0:
      print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
            f" Validation loss {eval_loss.item():.4f}")


In [203]:
linear_model = nn.Linear(5, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
  n_epochs = 3000,
  optimizer = optimizer,
  model = linear_model,
  loss_fn = nn.MSELoss(),
  train_data = train_data,
  train_price = train_price,
  eval_data = eval_data,
  eval_price = eval_price)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 1.5713, Validation loss 1.4073
Epoch 50, Training loss 0.4745, Validation loss 0.4925
Epoch 100, Training loss 0.4405, Validation loss 0.4543
Epoch 150, Training loss 0.4373, Validation loss 0.4482
Epoch 200, Training loss 0.4367, Validation loss 0.4462
Epoch 250, Training loss 0.4365, Validation loss 0.4453
Epoch 300, Training loss 0.4364, Validation loss 0.4449
Epoch 350, Training loss 0.4364, Validation loss 0.4447
Epoch 400, Training loss 0.4364, Validation loss 0.4446
Epoch 450, Training loss 0.4364, Validation loss 0.4445
Epoch 500, Training loss 0.4364, Validation loss 0.4445
Epoch 550, Training loss 0.4364, Validation loss 0.4445
Epoch 600, Training loss 0.4364, Validation loss 0.4444
Epoch 650, Training loss 0.4364, Validation loss 0.4444
Epoch 700, Training loss 0.4364, Validation loss 0.4444
Epoch 750, Training loss 0.4364, Validation loss 0.4444
Epoch 800, Training loss 0.4364, Validation loss 0.4444
Epoch 850, Training loss 0.4364, Validation loss 0.

In [204]:
seq_model = nn.Sequential(
            nn.Linear(1, 13),
            nn.Tanh(),
            nn.Linear(13, 1))

seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [205]:
[param.shape for param in seq_model.parameters()]

[torch.Size([13, 1]), torch.Size([13]), torch.Size([1, 13]), torch.Size([1])]

In [206]:
for name, param in seq_model.named_parameters():
  print(name, param.shape)

0.weight torch.Size([13, 1])
0.bias torch.Size([13])
2.weight torch.Size([1, 13])
2.bias torch.Size([1])


In [207]:
from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
  ('hidden_linear', nn.Linear(5, 8)),
  ('hidden_activation', nn.Tanh()),
  ('output_linear', nn.Linear(8, 5)),
]))

seq_model

Sequential(
  (hidden_linear): Linear(in_features=5, out_features=8, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=8, out_features=5, bias=True)
)

In [208]:
for name, param in seq_model.named_parameters():
  print(name, param.shape)  

hidden_linear.weight torch.Size([8, 5])
hidden_linear.bias torch.Size([8])
output_linear.weight torch.Size([5, 8])
output_linear.bias torch.Size([5])


In [209]:
seq_model.output_linear.bias

Parameter containing:
tensor([ 0.3377, -0.0539, -0.1557, -0.1014, -0.1906], requires_grad=True)

In [210]:
##train_price_fixed = train_price.clone().detach()
#train_price_fixed.shape
train_price = torch.tensor(train_price).squeeze(1)
train_price = torch.tensor(train_price).unsqueeze(1)
train_price.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


torch.Size([436, 1])

In [211]:
eval_price = torch.tensor(eval_price).squeeze(1)
eval_price = torch.tensor(eval_price).unsqueeze(1)
eval_price.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


torch.Size([109, 1])

In [212]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-5)

training_loop(
  n_epochs = 200,
  optimizer = optimizer,
  model = seq_model,
  loss_fn = nn.MSELoss(),
  train_data = train_data,
  train_price = train_price,
  eval_data = eval_data,
  eval_price = eval_price)

  #return F.mse_loss(input, target, reduction=self.reduction)
print('output', seq_model(eval_data))
print('answer', eval_price)
print('hidden', seq_model.hidden_linear.weight.grad)

Epoch 1, Training loss 1.2016, Validation loss 1.0584
Epoch 50, Training loss 1.2013, Validation loss 1.0581
Epoch 100, Training loss 1.2011, Validation loss 1.0579
Epoch 150, Training loss 1.2008, Validation loss 1.0577
Epoch 200, Training loss 1.2005, Validation loss 1.0574
output tensor([[ 0.5214,  0.0668, -0.0480,  0.1604, -0.0431],
        [ 0.5949, -0.4415, -0.1387, -0.1567, -0.0455],
        [ 0.5089, -0.2280, -0.4883, -0.3952, -0.3583],
        [ 0.3482, -0.1252,  0.1048, -0.4208, -0.2160],
        [ 0.1989, -0.0410, -0.1121, -0.4397, -0.2047],
        [ 0.3689,  0.1087,  0.0709, -0.3243, -0.3086],
        [ 0.3471, -0.1756,  0.1086, -0.4344, -0.2023],
        [ 0.1171,  0.0095, -0.4501, -0.6303, -0.4646],
        [ 0.1907,  0.1003, -0.1300, -0.3107, -0.2852],
        [ 0.6030, -0.2323,  0.2769, -0.0482,  0.0310],
        [ 0.4135, -0.3936, -0.3426, -0.1660,  0.0144],
        [ 0.2431, -0.1190, -0.6081, -0.4879, -0.3688],
        [-0.0120,  0.3412, -0.2849, -0.3462, -0.3681],
 

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([436, 1])) that is different to the input size (torch.Size([436, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([109, 1])) that is different to the input size (torch.Size([109, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
